Ce corpus vous est normalement fourni pour ce BE (CPE 19-20).

Les données ont été découpées : une recette par fichier. Il y a 220 recettes.
    
On suppose que les 220 documents de ce corpus est placé dans  "./data/recipes/{1, 2, ..., 220}.txt" (donnez le vôtre !)

# Exploration des données Textes

## Recettes Italiens

Origine : https://www.gutenberg.org/ebooks/24407 (public domain)

Les données dans : `./data/recipes/{1, 2, ..., 220}.txt`

Il y a  220 recettes (unerecette par fichier)

## I) Charger les données 

Chargement dans le dico `documents` 

En même temps, et pour faciliter les manipulations, les documents sont regroupés dans un STRING
:  `corpus_all_in_one` 

In [1]:
import os

data_folder = os.path.join('./', 'data', 'recipes')
all_recipe_files = [os.path.join(data_folder, fname)
                    for fname in os.listdir(data_folder)]

documents = {}
for recipe_fname in all_recipe_files:
    bname = os.path.basename(recipe_fname)
    recipe_number = os.path.splitext(bname)[0]
    with open(recipe_fname, 'r') as f:
        documents[recipe_number] = f.read()

corpus_all_in_one = ' '.join([doc for doc in documents.values()])

print("Nbr de docs: {}".format(len(documents)))
print("Taille Corpus : {}".format(len(corpus_all_in_one)))

Nbr de docs: 220
Taille Corpus : 161146


## 2) Tokenisation

Tokenisation est le process de découper les chaînes de caractères en une liste de tokens (lexemes, termes).

Un token peut être (since we speak English !)   :
- Words
- Phrases
- Punctuation
- Numbers
- Dates
- Currencies
- Hashtags
- ...?

In [3]:
from nltk.tokenize import word_tokenize

try:  # py3
    all_tokens = [t for t in word_tokenize(corpus_all_in_one)]
    print("traitement en python 3")
except UnicodeDecodeError:  # py27
    all_tokens = [t for t in word_tokenize(corpus_all_in_one.decode('utf-8'))]
    print("traitement en python 2")

print("Nombre Total des tokens: {}".format(len(all_tokens)))

traitement en python 3
Nombre Total des tokens: 33719


## 3) Quelques tokens fréquents

Comptage avec `collections.Counter`

On veut trouver :

- Le nb occurrences d'un mot dans le corpus  (et  Tf : Terme Frequency)
- Et éventuellement : dans combien de documents ce mot apparait 

### 3.1) Tf : per document term-frequancy 

In [5]:
# TF : Fréquences par document : les  tokens les plus fréquens par doc
from collections import Counter

print("No Document\t (Token, Nb_occ du token dans Document)")
Max_nb_lignes=10    # On affichera que 10 réponses
for  ind in documents.keys() :
    doc_tokens = [t for t in word_tokenize(documents[ind])]
    
    occs = Counter(doc_tokens).most_common(2)
    print(ind,"\t\t", occs)
        
    Max_nb_lignes-=1
    if Max_nb_lignes < 0 : break
 

No Document	 (Token, Nb_occ du token dans Document)
148 		 [('it', 7), ('.', 7)]
14 		 [('and', 10), ('.', 8)]
145 		 [(',', 8), ('and', 5)]
152 		 [('the', 5), (',', 4)]
50 		 [(',', 8), ('the', 6)]
192 		 [(',', 17), ('and', 9)]
210 		 [('the', 5), ('is', 3)]
40 		 [('the', 52), (',', 37)]
13 		 [('.', 10), ('the', 9)]
64 		 [(',', 10), ('the', 10)]
203 		 [(',', 15), ('the', 14)]


* Notons au passage que certains termes sont inutiles (cf. '.', 'the' répété, ....)

### 3.2) Total_Tf : Total trem-frequancy 

In [7]:
# Fréquences dans le corpus

from collections import Counter

total_term_frequency = Counter(all_tokens)
print("Token \t nb_occ TOTAL")
print('_'*30)
for word, freq in total_term_frequency.most_common(20):
    print("{}\t{}".format(word, freq))

Token 	 nb_occ TOTAL
______________________________
the	1933
,	1726
.	1568
and	1435
a	1076
of	988
in	811
with	726
it	537
to	452
or	389
is	337
(	295
)	295
be	266
them	248
butter	231
on	220
water	205
little	198


### 3.3) Df   (document frequency) : dans combien de documents un mot apparait-il ?
Les mots considérés sont les (20) mots les plus fréquents (utile poir __Idf__ puis __TfIdf__ plus bas)

In [8]:
document_frequency = Counter()

for recipe_number, content in documents.items():
    tokens = word_tokenize(content)
    unique_tokens = set(tokens)
    document_frequency.update(unique_tokens)

for word, freq in document_frequency.most_common(20):
    print("{}\t{}".format(word, freq))

.	220
and	220
,	219
(	218
)	218
the	217
in	215
a	210
of	210
with	203
it	167
or	165
to	165
is	145
salt	142
butter	137
on	136
be	133
put	126
water	125


## 4) Stop-words (mots usuels)

On constate que certains mots (fréquents) ne sont pas porteurs d'information. C'est par exemple le cas d'un déterminant, article, pronons, ...

On les appelle "__mots usuels__" (**stop-words**) 
* N.B. :  il n'y a pas de liste de mots usuels universelle (per lingua)

* N.B. : Il y a parfois des "regrets" : 
    - Si on supprimes les mot susuels des expressions telles que 
"to be or not to be", que deviendrions-nous alors !?

In [10]:
from nltk.corpus import stopwords
import string

print("* Les mot usuels (En) : \n", stopwords.words('english'))
print("* Nb mots usuels (En)", len(stopwords.words('english')))
print("* Ponctuations (En) : ", string.punctuation)

* Les mot usuels (En) : 
 ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own

### Et  en Français :

In [11]:
from nltk.corpus import stopwords
import string

print("* Les mot susuels (Fr) : \n", stopwords.words('french'))
print("* Nb mots usuels (Fr)", len(stopwords.words('french')))
print("* Ponctuations (Fr) : ", string.punctuation)

* Les mot susuels (Fr) : 
 ['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'aur

## Suppression des mots usuels (anglais) dans notre corpus

In [12]:
stop_list = stopwords.words('english') + list(string.punctuation)

all_tokens_no_stop = [token for token in all_tokens if token not in stop_list]

total_term_frequency_no_stop = Counter(all_tokens_no_stop)

# Les 20 mots les plus fréquents
for word, freq in total_term_frequency_no_stop.most_common(20):
    print("{}\t{}".format(word, freq))

butter	231
water	205
little	198
put	197
one	186
salt	185
fire	169
half	169
two	157
When	132
pepper	128
sauce	128
add	125
cut	125
piece	116
flour	116
The	111
saucepan	100
sugar	100
oil	99


Note : **When** et **The**   (W et T en majuscule)
- Pour l'instant, on les différencie !

### Interrogeons quelques fréquences (totales) avec __distinction__ des Min et Maj.

In [134]:
print(total_term_frequency_no_stop['olive'])
print(total_term_frequency_no_stop['olives'])
print(total_term_frequency_no_stop['Olive'])
print(total_term_frequency_no_stop['Olives'])
print(total_term_frequency_no_stop['OLIVE'])
print(total_term_frequency_no_stop['OLIVES'])

27
3
1
0
0
1


## 5) Normalisation du corpus
On appelle "Normalisation" les étapes de   Stemming / Lemmatisation, ... qui interviennent après le 
nettoyage du texte  (cf. opérations précédentes).

On remplace les termes par leur souche (racine, radical) où différentes conjugaisions d'un mots seront considérées comme identiques.
 
Actions:
- mettre en minuscule
- stemming
- mapping : American-to-British 
- mapping : synonym 


### 5.1) Stemming

In [13]:
# Après normalisation, on affiche les 20 les plus fréquents

from nltk.stem import PorterStemmer
all_tokens_lower = [t.lower() for t in all_tokens]
stemmer = PorterStemmer()

tokens_no_stop_stemmed_normalised = [stemmer.stem(t) for t in all_tokens_lower if t not in stop_list]

# Nouvelles fréquences
total_term_frequency_stemmed_no_stop_normalised = Counter(tokens_no_stop_stemmed_normalised)

for word, freq in total_term_frequency_stemmed_no_stop_normalised.most_common(20):
    print(f"{word}\t{freq}")

put	286
butter	245
salt	215
piec	211
one	210
water	209
cook	208
littl	198
cut	175
half	170
brown	169
fire	169
egg	163
two	162
add	160
boil	154
sauc	152
pepper	130
serv	128
remov	127


* A comparer par exemple avec les 4 termes ci-dessous obtenus avant la normalisation.
    - butter : 231     (vs. 245 ici)
    - water : 205      (vs. 209)
    - little : 198     (il a perdu son 'e')
    - salt : 185       (vs 215)

__Remarques__ : 
- Un stem n'est pas toujours un "mot" (qui apparaît dans un dico)
- Se rappeler que les opérations comme la mise en minuscule sont irrévocables 
- Au fait : Ici, on fait abstraction du "bon style" de programmation (Python). 

###  5.2) Lemmatisation
* Alternativement, on peut __lemmatiser__ les tokens (le terme résultat retenu doit figurer dans un dico)

* Remarque :  notez p. ex. les termes  <font color='red'>"piece", "little" </font>  ci-dessous. 
* Certains termes ne sont plus parmi les 20 plus fréquents.

In [14]:
# Comparer  les résultats de la Lemmatization à ceux du stemming
import nltk
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer()

tokens_lemmatized_no_stop_normalised = [lemmatizer.lemmatize(t) for t in all_tokens_lower if t not in stop_list]

# Nouvelles fréquences
total_term_frequency_lemmatized_no_stop_normalized = Counter(tokens_lemmatized_no_stop_normalised)

for word, freq in total_term_frequency_lemmatized_no_stop_normalized.most_common(20):
    print(f"{word}\t{freq}")

put	276
butter	243
piece	211
one	210
water	209
little	198
salt	197
half	173
fire	169
cut	166
egg	163
two	162
add	160
sauce	152
pepper	130
flour	123
sugar	116
brown	105
saucepan	101
onion	101


## 6) n-grams 

Lorsque nous sommes intéressés par  le contexte des mots (dans une phrase par exemple), on peut 
utiliser les 
__n-grams__ qui représente l'entourage d'un mot (ses voisins). 

Un n-gram est une séquence de *n* mots (grams = grammes)  adjacents.

On utilise généralement des bi-grams ou tri-grams..


### 6.1) Exemples de 2-grams de notre corpus

In [15]:
from nltk import ngrams

phrases = Counter(ngrams(all_tokens_lower, 2))
for phrase, freq in phrases.most_common(20):
    print("{}\t{}".format(phrase, freq))

('in', 'the')	175
('in', 'a')	172
('of', 'the')	153
('with', 'a')	142
('.', 'when')	131
('the', 'fire')	129
('on', 'the')	128
('with', 'the')	117
(',', 'and')	117
('salt', 'and')	113
('it', 'is')	109
('a', 'little')	107
('piece', 'of')	102
('and', 'a')	102
('of', 'butter')	94
('and', 'pepper')	87
('.', 'the')	85
('and', 'the')	84
('when', 'the')	82
('with', 'salt')	80


### 6.2) Les fréquences des 3-grams de notre corpus

In [16]:
phrases = Counter(ngrams(all_tokens_lower, 3))
for phrase, freq in phrases.most_common(20):
    print("{}\t{}".format(phrase, freq))

('on', 'the', 'fire')	90
('salt', 'and', 'pepper')	84
('piece', 'of', 'butter')	73
('a', 'piece', 'of')	63
('with', 'salt', 'and')	62
('.', 'when', 'the')	59
('in', 'a', 'saucepan')	45
('a', 'pinch', 'of')	45
('season', 'with', 'salt')	42
('the', 'fire', 'with')	41
('when', 'it', 'is')	39
('and', 'pepper', '.')	37
('through', 'a', 'sieve')	36
('complete', 'the', 'cooking')	34
('and', 'a', 'half')	33
('of', 'butter', ',')	27
('a', 'taste', 'of')	26
('and', 'when', 'it')	26
('it', 'on', 'the')	26
(',', 'salt', 'and')	25


### 6.3)  La fréquence des 2-grams après la suppression des  stop-words

On peut noter que la suppression des Stop-word   affecte les n-grams

Par ex, le triplet  "a pinch of salt" deviendra  "pinch salt"; ce qui affectera ses n-grams.

In [18]:
phrases = Counter(ngrams(all_tokens_no_stop, 2))

for phrase, freq in phrases.most_common(20):
    print("{}\t{}".format(phrase, freq))

('salt', 'pepper')	106
('piece', 'butter')	73
('grated', 'cheese')	55
('bread', 'crumbs')	34
('put', 'fire')	32
('tomato', 'sauce')	32
('complete', 'cooking')	31
('brown', 'stock')	29
('thin', 'slices')	29
('season', 'salt')	29
('olive', 'oil')	26
('low', 'fire')	25
('chopped', 'fine')	25
('boiling', 'water')	22
('little', 'pieces')	22
('half', 'ounces')	21
('lemon', 'peel')	18
('one', 'two')	18
('two', 'ounces')	18
('half', 'cooked')	18


### 6.4) La fréquence des 3-grams après  la suppression des  stop-words

Même remarque que ci-dessus : on peut noter uqe la suppression des Stop-word   affecte les n-grams

De même, le triplet  "a pinch of salt" deviendra  "pinch salt"  et aura d'autres 3-grams.

In [19]:
phrases = Counter(ngrams(tokens_no_stop_stemmed_normalised, 3))

for phrase, freq in phrases.most_common(20):
    print("{}\t{}".format(phrase, freq))

('season', 'salt', 'pepper')	57
('bread', 'crumb', 'ground')	13
('cut', 'thin', 'slice')	13
('tast', 'lemon', 'peel')	12
('sprinkl', 'bread', 'crumb')	11
('pinch', 'grate', 'chees')	11
('greas', 'butter', 'sprinkl')	10
('small', 'piec', 'butter')	10
('good', 'oliv', 'oil')	10
('crumb', 'ground', 'fine')	9
('half', 'inch', 'thick')	9
('cut', 'small', 'piec')	9
('cut', 'littl', 'piec')	9
('anoth', 'piec', 'butter')	9
('medium', 'size', 'onion')	9
('ounc', 'sweet', 'almond')	9
('piec', 'butter', 'brown')	9
('saucepan', 'piec', 'butter')	9
('littl', 'piec', 'butter')	8
('tomato', 'sauc', '12')	8


### 6.5) Même chose avec les tokens lemmatisés

In [20]:
total_freq_lemm_phrases = Counter(ngrams(tokens_lemmatized_no_stop_normalised, 3))

for phrase, freq in total_freq_lemm_phrases.most_common(20):
    print("{}\t{}".format(phrase, freq))

('season', 'salt', 'pepper')	44
('bread', 'crumb', 'ground')	13
('cut', 'thin', 'slice')	13
('taste', 'lemon', 'peel')	12
('pinch', 'grated', 'cheese')	11
('small', 'piece', 'butter')	10
('good', 'olive', 'oil')	10
('crumb', 'ground', 'fine')	9
('half', 'inch', 'thick')	9
('greased', 'butter', 'sprinkled')	9
('cut', 'small', 'piece')	9
('cut', 'little', 'piece')	9
('another', 'piece', 'butter')	9
('medium', 'sized', 'onion')	9
('ounce', 'sweet', 'almond')	9
('saucepan', 'piece', 'butter')	9
('little', 'piece', 'butter')	8
('tomato', 'sauce', '12')	8
('three', 'half', 'ounce')	8
('butter', 'salt', 'pepper')	7
